In [1]:
import argparse
from argparse import RawTextHelpFormatter
import numpy as np
import os

# keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Reshape, UpSampling2D, Conv2DTranspose, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import Adam


from absl import app

# logging
import wandb
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import gridspec, colors
import shutil
from glob import glob
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import entropy
from scipy.stats import norm, kstest

Using TensorFlow backend.


## Load models

In [2]:
models = "/home/aogorodnikov/aae"
wd = "/home/aogorodnikov/test"
batch = 16

In [3]:
def model_compile(models):
    '''
    Loads and compiles models
    :param models: directory with encoder, decoder and discriminator

    :return: compiled autoencoder and discriminator
    '''

    encoder = load_model(os.path.join(models, "encoder.h5"))
    decoder = load_model(os.path.join(models, "decoder.h5"))

    # get input and latent space shapes
    latent_dim = encoder.get_layer(index=-1).output_shape[1]
    input_dim = encoder.get_layer(index=0).input_shape[1:]

    # compile imported models into assembled autoencoder
    autoencoder_input = Input(shape=input_dim)
    autoencoder = Model(autoencoder_input, decoder(encoder(autoencoder_input)))
    autoencoder.compile(optimizer=Adam(lr=1e-4), loss="mean_squared_error", metrics=['accuracy'])

    return autoencoder, encoder, input_dim, latent_dim

In [4]:
autoencoder, encoder, input_dim, latent_dim = model_compile(models)

Instructions for updating:
Colocations handled automatically by placer.


/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
def mse_batch(data_x, data_y, input_dim):
    # TODO: docstring
    shape_1 = input_dim[0]
    shape_2 = input_dim[1:]
    reshape_x = np.reshape(data_x, (shape_1, np.prod(shape_2)))
    reshape_y = np.reshape(data_y, (shape_1, np.prod(shape_2)))
    return mse(reshape_x, reshape_y)

In [6]:
def anomaly_score(img_wd, batch, input_dim):
    # TODO: docsting
    data_loader = ImageDataGenerator(
        rescale=1. / 255,
        featurewise_center=True,
        featurewise_std_normalization=True,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_data = data_loader.flow_from_directory(
        img_wd,
        target_size=(input_dim[0], input_dim[0]),
        batch_size=batch,
        class_mode='input')
    
    batch_index = 0
    discriminator_batch_losses = []
    while batch_index <= train_data.batch_index:
        data = train_data.next()
        data_list = data[0]
        data_size = len(data_list)

        fake_latent = encoder.predict(data_list)
        discriminator_input = np.concatenate((fake_latent, np.random.randn(data_size, latent_dim) * 5.))
        discriminator_labels = np.concatenate((np.zeros((data_size, 1)), np.ones((data_size, 1))))
        discriminator_history = discriminator.evaluate(x=discriminator_input, y=discriminator_labels)

            
        batch_index = batch_index + 1
        discriminator_batch_losses.append(discriminator_history[0])
    
    
    
    
    ae_res = autoencoder.evaluate_generator(train_data)[0]
    adv_res = np.mean(discriminator_batch_losses)
    
    
    
    
    if draw:
        fig = plt.figure(figsize=(5 * len(samples), 5))
        gs = gridspec.GridSpec(1, len(samples))
        for i, sample in enumerate(samples):
            ax = plt.Subplot(fig, gs[i])
            ax.imshow((sample["image"] * 255.).astype("int"), cmap='gray')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_aspect('equal')
            ax.set_title(sample["title"])
            fig.add_subplot(ax)
        plt.show(block=False)
        
    
    print('Class: {}\nae_loss: {}\nadv_loss: {}'.format(cell_pheno, ae_res, adv_res))
    return [ae_res, adv_res]
    
        

In [6]:
data_loader = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    shear_range=0,
    zoom_range=0,
    horizontal_flip=False)
# load data
data_in = data_loader.flow_from_directory(
    wd,
    target_size=(input_dim[0], input_dim[0]),
    batch_size=batch,
    shuffle=False,
    class_mode='input')

Found 313 images belonging to 1 classes.


In [7]:
data_in.filepaths[:17]

['/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f01_0386.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f02_0304.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f02_0652.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f04_0256.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f04_0319.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f04_0330.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f05_0031.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f05_0302.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f06_0232.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f08_0316.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f09_0084.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f10_0346.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f11_0035.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f11_0037.tif',
 '/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f11_0208.tif',
 '/home/aogorodnikov/test

In [8]:
batch_index = 0

data = data_in.next()
data_list = data[0]
data_size = len(data_list)

ae_pred = autoencoder.predict_on_batch(data_list)
fake_latent = encoder.predict(data_list)

/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [9]:
lol = [mse_batch(data_list[x], ae_pred[x], input_dim) for x in range(len(ae_pred))]

In [10]:
def kld_batch(latent_x, latent_dim):
    '''
    Function to compute adversarial loss
    :param latent_x: encoded latent space
    :return: KL divergence b/w prior
    '''
    prior = np.random.randn(latent_dim) * 5.
    return entropy(norm.pdf(latent_x), norm.pdf(prior))

In [11]:
shit  = [kld_batch(x, latent_dim) for x in fake_latent]

In [12]:
list(zip(data_in.filepaths[:17],lol, shit))

[('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f01_0386.tif',
  0.006219174,
  9.34047869459597),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f02_0304.tif',
  0.0023011477,
  8.538530194657401),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f02_0652.tif',
  0.0029112056,
  6.201995170506212),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f04_0256.tif',
  0.0029931194,
  9.753694729578307),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f04_0319.tif',
  0.0025848555,
  3.068893060820288),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f04_0330.tif',
  0.0039777756,
  8.490175263068908),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f05_0031.tif',
  0.0007974663,
  5.814659138153534),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f05_0302.tif',
  0.0022562735,
  13.935966666544264),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f06_0232.tif',
  0.0014631412,
  11.783931993434004),
 ('/home/aogorodnikov/test/cytokinesis/Pt04_r01c01_f08_0316.tif'

In [30]:
np.mean(lol)

0.0038870783

In [7]:
data = data_in.next()
data_list = data[0]
data_size = len(data_list)

/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [11]:
autoencoder.evaluate_generator(data_in)[0]

0.005301421973854303

## Notes  
We create a function that takes a single class (cellular phenotype) and test reconstruction loss for autoencoder and discriminator